<a href="https://colab.research.google.com/github/Shuraimi/fastai_DL_course/blob/main/10_nlp_clean_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 300.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 11.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires py

In [ ]:
#hide
from fastbook import *
from IPython.display import display,HTML

# NLP Deep Dive: RNNs

## Text Preprocessing

### Tokenization

### Word Tokenization with fastai

The code below imports everything from `text.all` and then load the IMDb dataset using `untar_data`

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

Similar to `get_image_files` to get all images from a path for CV task, we use `get_text_files` to get all text files from a path and also specify the folders to restrict the search of all available docs in the path.

In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

We then open a file i.e. the first and then read it. We then grab the first few characters of the doc/file to view it easily.

In [ ]:
txt = files[0].open().read(); txt[:75]

'I saw the film today and was mighty impressed. The film captured the buzz i'

To tokenize the files, we use SpacyTokenizer or use WordTokenizer by fastai which refers to the default tokenizer in fastai.

Then grab the first element of collection using `first` and then apply spacy.

Then use coll_repr to see the n number of tokens and it also gives the total number of tokens in text.

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#200) ['I','saw','the','film','today','and','was','mighty','impressed','.','The','film','captured','the','buzz','in','the',"'",'80s','when','heavy','metal','became','the','biggest','thing','going','.','It','was'...]


The tokenization rules are powerful enough that it can understand that the dot after U and S don't need to be separated.

In [ ]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [ ]:
spacy(['The U.S. dollar $1 is $1.00.'])

<generator object SpacyTokenizer.__call__.<locals>.<genexpr> at 0x7e0763dc0900>

fastai's Tokenizer has few more options and info than the WordTokenizer i.e. it adds special tokens

In [ ]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#231) ['xxbos','i','saw','the','film','today','and','was','mighty','impressed','.','xxmaj','the','film','captured','the','buzz','in','the',"'",'80s','when','heavy','metal','became','the','biggest','thing','going','.','xxmaj'...]


Get a list of all rules to see methods how the special token are added.

In [ ]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [ ]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

### Subword Tokenization

We open and read the first 2000 files and convert it to a L type

In [ ]:
txts = L(o.open().read() for o in files[:2000])

In [ ]:
len(txts)

2000

In [ ]:
type(txts)

fastcore.foundation.L

In [ ]:
txts[0]

"I saw the film today and was mighty impressed. The film captured the buzz in the '80s when heavy metal became the biggest thing going. It was good to see Iron Maiden and Judas Priest contrasted to current bands such as Slipknot to show how the spirit of the genre is thriving. Tony Iommi, Dee Snider and Ronnie James Dio all give standout interviews with Dio's digs at Gene Simmons being especially revealing. It was also great to see '80s curiosities Accept and Quebec's own Voivod being represented. The segment contrasting the grunting, leather-clad bands such as Man O War with the lace-wearing bands such as Poison and Cinderella was unexpected and fascinating. Showing the closeted Rob Halford performing in full cruising gear for a rabid hetero audience was quite poignant. I especially like how the film stresses how the music let its fans dream, cope and find solace. Those like myself, whose interest in metal may have flagged in the intervening years would do well to view it and remind t

We then define a function to perform subword tokenization and call the `setup` and pass it txts so that it can identify the most commonly occuring substrings or group of letters in words in order to tokenize.

To this function, we also pass the vocab size.

In [ ]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

We call the subword function and pass 1000 as the vocab size to get the subwords of txt given.

In [ ]:
subword(1000)

"▁I ▁saw ▁the ▁film ▁to day ▁and ▁was ▁might y ▁imp re s s ed . ▁The ▁film ▁ca p t ur ed ▁the ▁b u z z ▁in ▁the ▁ ' 8 0 s ▁when ▁he a v y"

Now we choose a smaller vocab size and this will give us more tokens per sentence

In [ ]:
subword(200)

'▁I ▁ s a w ▁the ▁film ▁to d a y ▁and ▁was ▁ m i g h t y ▁ i m p re s s ed . ▁The ▁film ▁ca p t ur ed ▁the ▁b u z'

And a larger token size will give us fewer tokens per sentence as most of the words remain as it.

_ represents that the original text has space at the position.

In [ ]:
subword(10000)

"▁I ▁saw ▁the ▁film ▁today ▁and ▁was ▁might y ▁impressed . ▁The ▁film ▁captured ▁the ▁buzz ▁in ▁the ▁'80 s ▁when ▁heavy ▁metal ▁became ▁the ▁bigge st ▁thing ▁going . ▁It ▁was ▁good ▁to ▁see ▁I ron ▁Maid en ▁and"


### Numericalization with fastai

There are two steps:-

1. Make a list of all possible unique words(i.e. the vocab)
2. Replace the tokens with their index in vocab

This is tokenizing the txt(which contains a few characters of sentence/first file) using Tokenizer

In [ ]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#231) ['xxbos','i','saw','the','film','today','and','was','mighty','impressed','.','xxmaj','the','film','captured','the','buzz','in','the',"'",'80s','when','heavy','metal','became','the','biggest','thing','going','.','xxmaj'...]


Now we want to tokenize first 200 docs using the .map(tkn) for each of 200 files

In [ ]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#231) ['xxbos','i','saw','the','film','today','and','was','mighty','impressed'...]

Now we instantiate Numericalize and call the setup method just like we did for subword tokenizer. Here we call setup so that numericalize can go through the tokens to grab unique words from it and make the vocab

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2040) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','to','of','is','in','it','i'...]"

This vocab has all special words at the beginning followed by each word with a frequency of one.

Now we convert the tokenized text into numbers anf grab the first 20

In [ ]:
nums = num(toks)[:20]; nums

TensorText([   2,   19,  173,    9,   32,  593,   12,   24,    0, 1498,   11,    8,    9,   32,  832,    9,    0,   17,    9,   88])

Then we map these numbers to respective token using the vocab which has its index

In [ ]:
' '.join(num.vocab[o] for o in nums)

"xxbos i saw the film today and was xxunk impressed . xxmaj the film captured the xxunk in the '"

### Putting Our Texts into Batches for a Language Model

The code below takes a stream, tokenizes it and then converts into an array. Tokens are from start index to end index such that these mini-streams are contiguous and we get 6 such arrays.

We then convert it into a DataFrame and display as HTML.

In [ ]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


This 👇 is also the same thing but the batches are from 0-5,15-20 and so on i.e. we get the first 5 tokens from the 15 token sentence.

In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


This is also same as before but here we get the batches with tokens from 5-10,20-25 and so on i.e. we get the middle part of 15 tokens sentence.

In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


This is also the same thing but here we get batches with tokens from 10-15,25-30 and so on i.e. we get the last part of the 15 token sentence.

In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


Now we numericalize the toks200 i.e. the tokenized 200 files.

In [ ]:
nums200 = toks200.map(num)

In [ ]:
nums200[0]

TensorText([   2,   19,  173,    9,   32,  593,   12,   24,    0, 1498,   11,    8,    9,   32,  832,    9,    0,   17,    9,   88, 1499,   62,  979,    0, 1181,    9,  833,  138,  185,   11,    8,
              18,   24,   79,   14,   87,    8,    0,    8,    0,   12,    8,    0,    8, 1182,    0,   14, 1500, 1183,  174,   28,    8,    0,   14,  133,   95,    9,    0,   15,    9,  410,   16,
               0,   11,    8, 1501,    8,    0,   10,    8,    0,    8,    0,   12,    8,    0,    8,  980,    8,    0,   45,  206,    0,    0,   30,    8,    0,   23,    0,   50,    8,    0,    8,
               0,  123,  201,    0,   11,    8,   18,   24,  103,   99,   14,   87,   88, 1499,    0,    8,  834,   12,    8,    0,   23,  202,    8,    0,  123,    0,   11,    8,    9,  981,    0,
               9,    0,   10,    0,   26,    0, 1183,  174,   28,    8,  135, 1502,    8,  180,   30,    9,    0,   26, 1184, 1183,  174,   28,    8,    0,   12,    8,    0,   24, 1503,   12,    0,
          

As a recap,

at each epoch,
- we shuffle the documents
- concatenated them into stream of tokens
- cut them down into batches which consists of fixed consecutive mini-streams.
- then our model will read the mini-streams in order

All these are handled behind the scenes by the `LMDataLoader`. We do this by first applying the numericalize to tokenized text and then pass it to LMDataLoader.

In [ ]:
dl = LMDataLoader(nums200)

We then grab the first batch and check shapes

In [ ]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

Getting the independent variables of first row and grabbing few tokens

In [ ]:
' '.join(num.vocab[o] for o in x[0][:20])

"xxbos i saw the film today and was xxunk impressed . xxmaj the film captured the xxunk in the '"

The same for dependent is same but with an offset of one token

In [ ]:
' '.join(num.vocab[o] for o in y[0][:20])

"i saw the film today and was xxunk impressed . xxmaj the film captured the xxunk in the ' 80s"

## Training a Text Classifier

### Language Model Using DataBlock

In order to fine-tune a pretrained language model, we need to create a DataBlock and pass it the parameters like we did for image classification.

fastai does tokenization and numericalization automatically for us when we pass TextBlock as blocks.

First we use `partial` and create a new function which is same as `get_text_files` but the folders are fixed.

We then create a DataBlock and pass it TextBlock as blocks and set is_lm=True which takes the next word as labe and then create DataLoaders and pass the path of data and path where it should store the tokenized inputs.

In [ ]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [ ]:
dls_lm.show_batch(max_n=2)

### Fine-Tuning the Language Model

We then create a language model learner which takes the DataLoaders we created and use the RNNs architecture AWD_LSTM. The embeddings of pretrained model are merged with random embeddings of words that weren't in the pretrained vocab. This is automatically handled by language_model_leaner. These embeddings are fed to the RNNs.

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

To fine-tune, we use fit_one_cycle because it allows us to save intermediate results and we can manually fine-tune the model by gradually unfreezing few layers at a time.

In [ ]:
learn.fit_one_cycle(1, 2e-2)

### Saving and Loading Models

Save model

In [ ]:
learn.save('1epoch')

Load model

In [ ]:
learn = learn.load('1epoch')

Unfreeze layers of the model and fine-tune

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

Save the body which is called as encoder in NLP, leaving the task specific head.

In [ ]:
learn.save_encoder('finetuned')

So now we've completed fine-tuning the language model to the IMDb corpus.

### Text Generation

We'll see how good the model is at predicting the next word of movie reviews.

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

### Creating the Classifier DataLoaders

Now we will create a DataLoaders for classifier to fine-tune the language model for classification task.

We pass it the vocab of the fine-tuned language model on IMBd corpus so that it uses the same correspondence of token to index. And also that the embedding of pretrained model won't make sense to this classifier model and fine-tuning will not be useful.


Sorting and padding are automatically doen by the DataBlock API when we use TextBlock.

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

In [ ]:
nums_samp = toks200[:10].map(num)

In [ ]:
nums_samp.map(len)

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

Creating a text Classifier

In [ ]:
learn = learn.load_encoder('finetuned')

Now we load the encoder for it to fine-tuned.

### Fine-Tuning the Classifier

Used for fine-tuning the classfier

In [ ]:
learn.fit_one_cycle(1, 2e-2)

Fine-tuning after freezing all layers until specified layers

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

## Disinformation and Language Models

## Conclusion

## Questionnaire

1. What is "self-supervised learning"?
1. What is a "language model"?
1. Why is a language model considered self-supervised?
1. What are self-supervised models usually used for?
1. Why do we fine-tune language models?
1. What are the three steps to create a state-of-the-art text classifier?
1. How do the 50,000 unlabeled movie reviews help us create a better text classifier for the IMDb dataset?
1. What are the three steps to prepare your data for a language model?
1. What is "tokenization"? Why do we need it?
1. Name three different approaches to tokenization.
1. What is `xxbos`?
1. List four rules that fastai applies to text during tokenization.
1. Why are repeated characters replaced with a token showing the number of repetitions and the character that's repeated?
1. What is "numericalization"?
1. Why might there be words that are replaced with the "unknown word" token?
1. With a batch size of 64, the first row of the tensor representing the first batch contains the first 64 tokens for the dataset. What does the second row of that tensor contain? What does the first row of the second batch contain? (Careful—students often get this one wrong! Be sure to check your answer on the book's website.)
1. Why do we need padding for text classification? Why don't we need it for language modeling?
1. What does an embedding matrix for NLP contain? What is its shape?
1. What is "perplexity"?
1. Why do we have to pass the vocabulary of the language model to the classifier data block?
1. What is "gradual unfreezing"?
1. Why is text generation always likely to be ahead of automatic identification of machine-generated texts?

### Further Research

1. See what you can learn about language models and disinformation. What are the best language models today? Take a look at some of their outputs. Do you find them convincing? How could a bad actor best use such a model to create conflict and uncertainty?
1. Given the limitation that models are unlikely to be able to consistently recognize machine-generated texts, what other approaches may be needed to handle large-scale disinformation campaigns that leverage deep learning?